In [ ]:

%run includes/includes.ipynb


In [ ]:

# show header

owner = owner.value
repo = repo.value
header_text = f"# Sprint {sprint_number.value}\n"
header_text += f"#### {start_date.value} to {end_date.value}\n"
display(Markdown(header_text))

In [ ]:
%%stackql
SELECT 
SUBSTRING(sha, 1, 7) as sha, 
JSON_EXTRACT(commit, '$$.author.name') as author,  
JSON_EXTRACT(commit, '$$.author.date') as date
FROM github.repos.commits
WHERE owner = '$owner' AND repo = '$repo'

In [ ]:
%%stackql --no-display
SELECT login, contributions  FROM github.repos.contributors WHERE owner = '$owner' AND repo = '$repo'

In [ ]:
show_contributions(stackql_df)

In [ ]:
%%stackql
SELECT 
published_at,
target_commitish,
tag_name
FROM github.repos.releases WHERE owner = '$owner' AND repo = '$repo'